In [1]:
import pandas as pd
import json

In [2]:
#---------------- INTO CSV ----------------#
def into_csv(data, write_file='data.csv'):
    f = open(write_file, "w+")
    f.close()
    data.to_csv(write_file, sep=';')
    print(write_file, '\tdone')

In [50]:
f = open('Dict_names.txt')
true_names_dict = json.load(f)
f.close()
print(true_names_dict["Олешинська"])

Олешин


In [5]:
#---------------- PROPERTY TABLE ----------------#
def get_true_name(name):
    f = open(str(INN)+'.txt')
    true_names_dict = json.load(f)
    f.close()

    # true_names_dict = {'Чанизька': 'Чаниж',
    #                   'Артемівська': 'Артемівка'}
    
    try:
        true_name = true_names_dict[name]
    except:
        true_name = name
    
    return true_name



def get_area(val):
    import re
    lst = re.findall(r'Площа \(га\): \d+[,.]?\d+|льна площа \(кв\.м\): \d+[,.]?\d+', val)
    
    val = [re.findall(r'\d+[,.]?\d+', item) for item in lst]
    val = "".join([item for sublist in val for item in sublist])
    
    return val


def get_address(val):
    import re
    # Область
    obl = re.findall(r'\w+(?=\sобл\.)', val)
    obl = "".join([item for sublist in obl for item in sublist])
    
    # Район
    rai = re.findall(r'\w+(?=\sр\.)', val)
    rai = "".join([item for sublist in rai for item in sublist])
    
    # місто/село/сільська рада
    nasP = re.findall(r'(?<=\bрада\.)\s?\w+\b', val)
    if nasP:
        nasP = "".join([item for sublist in nasP for item in sublist])
        nasP = re.sub(r'\A\s', '', nasP)
        nasP = get_true_name(nasP)
    else:
        nasP = re.findall(r'(?<=\bм\.)\s?\w+\b' + r'|(?<=\bс\.)\s?\w+\b', val)
        nasP = "".join([item for sublist in nasP for item in sublist])
    nasP = re.sub(r'\A\s', '', nasP)
    
    # тип вулиці
    str_type = re.findall(r'вулиця|провулок|проспект|бульвар|масив|квартал|площа', val)
    if not str_type:
        str_type = 'інформація відсутня'
    else:
        str_type = "".join([item for sublist in str_type for item in sublist])
    
    # вулиця/провулок
    street = re.findall(r'(?<=\bвулиця)\s?[\w\s.]+(?=,)' \
                        + r'|(?<=\bпровулок)\s?[\w\s.]+(?=,)' \
                        + r'|(?<=\bпроспект)\s?[\w\s.]+(?=,)' \
                        + r'|(?<=\bбульвар)\s?[\w\s.]+(?=,)' \
                        + r'|(?<=\bмасив)\s?[\w\s.]+(?=,)' \
                        + r'|(?<=\bквартал)\s?[\w\s.]+(?=,)' \
                        + r'|(?<=\bплоща)\s?[\w\s.]+(?=,)', val)
    if not street:
        street = 'інформація відсутня'
    else:
        street = "".join([item for sublist in street for item in sublist])
    street = re.sub(r'\A\s', '', street)
    
    # будинок
    houseNum = re.findall(r'(?<=\bбудинок)\s?[\w-]+', val)
    if not houseNum:
        houseNum = 'б/н'
    else:
        houseNum = "".join([item for sublist in houseNum for item in sublist])
    houseNum = re.sub(r'\A\s', '', houseNum)
    
    # квартира
    appNum = re.findall(r'(?<=\bквартира)\s?[\w-]+', val)
    appNum = "".join([item for sublist in appNum for item in sublist])
    appNum = re.sub(r'\A\s', '', appNum)
    
#     print(obl, '\t', rai, '\t', nasP, '\t', str_type, '\t', street, '\t', houseNum, '\t', appNum)
    
    return [obl, rai, nasP, str_type, street, houseNum, appNum]


def get_obj_type(val):
    import re
    # Область
    obj_type = ''
    if re.search(r'земельна', val) is not None:
        obj_type = 'земля'
    elif re.search(r'будинок', val) is not None:
        obj_type = 'будинок'
    elif re.search(r'квартира', val) is not None:
        obj_type = 'квартира'
    else: obj_type = 'комерція'
    #print(obj_type)

    return obj_type

In [6]:
files = [i for _, i in pd.read_csv('_INN.txt', delimiter='\n')['INN'].items()]

In [ ]:
# --------------- MAIN FOR RAW CSV --------------- #
for INN in files:
    f = open(str(INN)+'.txt')
    raw_data = json.load(f)
    f.close()

    data = pd.DataFrame(raw_data["answer"]["items"], columns=['name', 'value'])

    # print(pd.DataFrame(dict_data))
    #
    into_csv(data, 'results/'+str(INN)+'.csv')
print('all done')

In [7]:
# ------------------ MAIN PROPERTY TABLE ------------------ #
def prepare_data(file='2122810826.txt'):
    f = open(file)
    #f = open('1732712205.txt')
    raw_data = json.load(f)
    f.close()
    data = raw_data["answer"]["items"]
    for i in range(len(data)):
        if data[i]['value'] is None:
            data[i]['value'] = 'null'

    return pd.DataFrame(data, columns=['name', 'value'])


def get_properties(data):
    last_index = 9
    properties = pd.DataFrame(columns=['name', 'value'])
    DateOfFile = ''

    for index, row in enumerate(data.itertuples()):
    #     if index<last_index:
    #         continue
        df = pd.DataFrame([[row[1], row[2]]], columns=['name', 'value'])
        properties = properties.append(df, ignore_index = True)

        if row[1] == 'Дата, час формування:':
            DateOfFile = row[2]
        if row[1] == 'ВІДОМОСТІ З РЕЄСТРУ ПРАВ ВЛАСНОСТІ НА НЕРУХОМЕ МАЙНО':
            break

#     for index, row in properties.iterrows():
#         print(row['name'], row['value'])
    
    return properties, DateOfFile

    
    
def form_property_table(INN, properties, DateOfFile, file='results/FullProperty.txt', mode='a'):
    table_col = ['index', 'Опис об’єкта', 'Цільове призначення', 'Адреса', 'Кадастровий номер', 'Вид', 'Об’єкт нерухомого майна',
            'LMDescr', 'Область', 'Район', 'Нас. пункт', 'Тип вулиці', 'Вулиця', 'Будинок', 'Квартира',
            'Площа', 'Дата витягу', 'INN',
            'PropertyRights']
    obj = 0
    table = pd.DataFrame(columns=table_col)

    #temp = pd.DataFrame(columns=table_col)
    for index, row in properties.iterrows():
        if obj != 0:
            if row['value'] == 'null':
                obj = 0

            elif row['name'] == 'Об’єкт нерухомого майна:':
                table = table.append({'index': obj, 'Об’єкт нерухомого майна': row['value']}, ignore_index = True)
                table.loc[table['index'] == obj, 'Вид'] = get_obj_type(row['value'])
                table.loc[table['index'] == obj, 'LMDescr'] = row['value']
                table.loc[table['index'] == obj, 'Дата витягу'] = DateOfFile.split(' ')[0]
                table.loc[table['index'] == obj, 'INN'] = INN
                table.loc[table['index'] == obj, 'PropertyRights'] = '|'

            elif row['name'] == 'Кадастровий номер:':
                table.loc[table['index'] == obj, 'Кадастровий номер'] = row['value']
                table.loc[table['index'] == obj, 'LMDescr'] += ". КН: " + row['value']
            elif row['name'] == 'Опис об’єкта:':
                table.loc[table['index'] == obj, 'Опис об’єкта'] = row['value']
                table.loc[table['index'] == obj, 'LMDescr'] += ". " + row['value']
                table.loc[table['index'] == obj, 'Площа'] = get_area(row['value'])
            elif row['name'] == 'Цільове призначення:':
                table.loc[table['index'] == obj, 'Цільове призначення'] = row['value']
            elif row['name'] == 'Адреса:':
                table.loc[table['index'] == obj, 'Адреса'] = row['value']
                table.loc[table['index'] == obj, table_col[8:15]] = get_address(row['value'])



        if row['name'] == 'Актуальна інформація про об’єкт нерухомого майна':
            obj = index

#     print(table.columns[7:15])
#     print(table[table_col[4:15]])
#     print("\n".join([i for i in table['LMDescr']]))

    for index, row in table.iterrows():
        if row['Вид'] == 'земля':
            row['Квартира'] = row['Кадастровий номер']
        
    f = open(file, mode)
    f.close()
    table.to_csv(file, sep=';', columns=table_col[5:18], mode=mode)
    print(file, '\tdone')

In [9]:
# -------------------------------- MAIN -------------------------------- #
for INN in files:
    print('INN: ', INN)
    data = prepare_data(str(INN)+'.txt')
    properties, DateOfFile = get_properties(data)
    form_property_table(INN, properties, DateOfFile)
#     data = prepare_data('2122810826.txt')

INN:  3161227017
results/FullProperty.txt 	done
INN:  2581412945
results/FullProperty.txt 	done
INN:  3032501888
results/FullProperty.txt 	done
INN:  2933215591
results/FullProperty.txt 	done
INN:  2636709621
results/FullProperty.txt 	done
INN:  2087001433
results/FullProperty.txt 	done
INN:  2163000959
results/FullProperty.txt 	done
INN:  2822308891
results/FullProperty.txt 	done
INN:  2866921321
results/FullProperty.txt 	done
INN:  2988918405
results/FullProperty.txt 	done
INN:  2815308907
results/FullProperty.txt 	done
INN:  1962400068
results/FullProperty.txt 	done
INN:  1732712205
results/FullProperty.txt 	done
INN:  2122810826
results/FullProperty.txt 	done


In [8]:
# очистка файла
f = open('results/FullProperty.txt', 'w+')
f.close()

In [ ]:
# очистка файла
f = open('results/FullProperty.txt', 'w')
f.close()

In [4]:
files = [i for _, i in pd.read_csv('_INN.txt', delimiter='\n')['INN'].items()]

In [4]:
# --------------- MAIN FOR RAW CSV --------------- #
for INN in files:
    f = open(str(INN)+'.txt')
    raw_data = json.load(f)
    f.close()

    data = pd.DataFrame(raw_data["answer"]["items"], columns=['name', 'value'])

    # print(pd.DataFrame(dict_data))
    #
    into_csv(data, 'results/'+str(INN)+'.csv')
print('all done')

results/3161227017.csv 	done
results/2581412945.csv 	done
results/3032501888.csv 	done
results/2933215591.csv 	done
results/2636709621.csv 	done
results/2087001433.csv 	done
results/2163000959.csv 	done
results/2822308891.csv 	done
results/2866921321.csv 	done
results/2988918405.csv 	done
results/2815308907.csv 	done
results/1962400068.csv 	done
results/1732712205.csv 	done
results/2122810826.csv 	done
all done


In [5]:
# ------------------ MAIN PROPERTY TABLE ------------------ #
def prepare_data(file='2122810826.txt'):
    f = open(file)
    #f = open('1732712205.txt')
    raw_data = json.load(f)
    f.close()
    data = raw_data["answer"]["items"]
    for i in range(len(data)):
        if data[i]['value'] is None:
            data[i]['value'] = 'null'

    return pd.DataFrame(data, columns=['name', 'value'])


def get_properties(data):
    last_index = 9
    properties = pd.DataFrame(columns=['name', 'value'])

    for index, row in enumerate(data.itertuples()):
    #     if index<last_index:
    #         continue
        df = pd.DataFrame([[row[1], row[2]]], columns=['name', 'value'])
        properties = properties.append(df, ignore_index = True)

        if row[1] == 'Дата, час формування:':
            DateOfFile = row[2]
        if row[1] == 'ВІДОМОСТІ З РЕЄСТРУ ПРАВ ВЛАСНОСТІ НА НЕРУХОМЕ МАЙНО':
            break

#     for index, row in properties.iterrows():
#         print(row['name'], row['value'])
    
    return properties, DateOfFile

    
    
def form_property_table(INN, properties, DateOfFile, file='results/FullProperty.txt', mode='a'):
    table_col = ['index', 'Опис об’єкта', 'Цільове призначення', 'Адреса', 'Кадастровий номер', 'Вид', 'Об’єкт нерухомого майна',
            'LMDescr', 'Область', 'Район', 'Нас. пункт', 'Тип вулиці', 'Вулиця', 'Будинок', 'Квартира',
            'Площа', 'Дата витягу', 'INN',
            'PropertyRights']
    obj = 0
    table = pd.DataFrame(columns=table_col)

    #temp = pd.DataFrame(columns=table_col)
    for index, row in properties.iterrows():
        if obj != 0:
            if row['value'] == 'null':
                obj = 0

            elif row['name'] == 'Об’єкт нерухомого майна:':
                table = table.append({'index': obj, 'Об’єкт нерухомого майна': row['value']}, ignore_index = True)
                table.loc[table['index'] == obj, 'Вид'] = get_obj_type(row['value'])
                table.loc[table['index'] == obj, 'LMDescr'] = row['value']
                table.loc[table['index'] == obj, 'Дата витягу'] = DateOfFile.split(' ')[0]
                table.loc[table['index'] == obj, 'INN'] = INN
                table.loc[table['index'] == obj, 'PropertyRights'] = '|'

            elif row['name'] == 'Кадастровий номер:':
                table.loc[table['index'] == obj, 'Кадастровий номер'] = row['value']
                table.loc[table['index'] == obj, 'LMDescr'] += ". КН: " + row['value']
            elif row['name'] == 'Опис об’єкта:':
                table.loc[table['index'] == obj, 'Опис об’єкта'] = row['value']
                table.loc[table['index'] == obj, 'LMDescr'] += ". " + row['value']
                table.loc[table['index'] == obj, 'Площа'] = get_area(row['value'])
            elif row['name'] == 'Цільове призначення:':
                table.loc[table['index'] == obj, 'Цільове призначення'] = row['value']
            elif row['name'] == 'Адреса:':
                table.loc[table['index'] == obj, 'Адреса'] = row['value']
                table.loc[table['index'] == obj, table_col[8:15]] = get_address(row['value'])



        if row['name'] == 'Актуальна інформація про об’єкт нерухомого майна':
            obj = index

#     print(table.columns[7:15])
#     print(table[table_col[4:15]])
#     print("\n".join([i for i in table['LMDescr']]))

    for index, row in table.iterrows():
        if row['Вид'] == 'земля':
            row['Квартира'] = row['Кадастровий номер']
        
    f = open(file, mode)
    f.close()
    table.to_csv(file, sep=';', columns=table_col[5:18], mode=mode)
    print(file, '\tdone')

In [7]:
# -------------------------------- MAIN -------------------------------- #
for INN in files:
    print('INN: ', INN)
    data = prepare_data(str(INN)+'.txt')
    properties, DateOfFile = get_properties(data)
    form_property_table(INN, properties, DateOfFile)
#     data = prepare_data('2122810826.txt')

INN:  3161227017
results/FullProperty.txt 	done
INN:  2581412945
results/FullProperty.txt 	done
INN:  3032501888
results/FullProperty.txt 	done
INN:  2933215591
results/FullProperty.txt 	done
INN:  2636709621
results/FullProperty.txt 	done
INN:  2087001433
results/FullProperty.txt 	done
INN:  2163000959
results/FullProperty.txt 	done
INN:  2822308891
results/FullProperty.txt 	done
INN:  2866921321
results/FullProperty.txt 	done
INN:  2988918405
results/FullProperty.txt 	done
INN:  2815308907
results/FullProperty.txt 	done
INN:  1962400068
results/FullProperty.txt 	done
INN:  1732712205
results/FullProperty.txt 	done
INN:  2122810826
results/FullProperty.txt 	done


In [63]:
properties, DateOfFile = get_properties(data)

Інформація з Державного реєстру речових прав на нерухоме майно та Реєстру прав власності на нерухоме майно, Державного реєстру Іпотек, Єдиного реєстру заборон відчуження об’єктів нерухомого майна щодо суб’єкта null
Номер інформаційної довідки: 210239418
Дата, час формування: 25.05.2020 20:59:21
Підстава формування інформаційної довідки: пошук через веб-сайт: ШВАЛЯ АНДРІЙ ВАСИЛЬОВИЧ
Параметри запиту null
Пошук в Державному реєстрі речових прав на нерухоме майно про: права власності, інші речові права, іпотеки, обтяження
Тип особи: фізична особа
РНОКПП: 2122810826
Пошук за: частковим співпадінням
ВІДОМОСТІ З ДЕРЖАВНОГО РЕЄСТРУ РЕЧОВИХ ПРАВ НА НЕРУХОМЕ МАЙНО null
Актуальна інформація про об’єкт нерухомого майна null
Реєстраційний номер об’єкта нерухомого майна: 1358894446206
Об’єкт нерухомого майна: земельна ділянка
Кадастровий номер: 4620687000:15:000:0061
Опис об’єкта: Площа (га): 1.2668
Цільове призначення: для ведення товарного сільськогосподарського виробництва
Адреса: Львівська обл.

In [81]:
form_property_table(properties, DateOfFile)

                                             LMDescr       Область  \
0  земельна ділянка. КН: 4620687000:15:000:0061. ...     Львівська   
1  земельна ділянка. КН: 4620687000:12:000:0013. ...     Львівська   
2  земельна ділянка. КН: 4620687000:15:000:0060. ...     Львівська   
3  житловий будинок, об'єкт житлової нерухомості:...     Львівська   
4  інше житловий будинок садибного типу, об'єкт ж...  Закарпатська   

          Район Нас. пункт           Тип вулиці               Вулиця Будинок  \
0       Буський      Чаниж  інформація відсутня  інформація відсутня     б/н   
1       Буський      Чаниж  інформація відсутня  інформація відсутня     б/н   
2       Буський      Чаниж  інформація відсутня  інформація відсутня     б/н   
3       Буський      Чаниж               вулиця   ім. Семена Лісовця     200   
4  Берегівський       Бене               вулиця               Ракоці     175   

  Квартира  
0           
1           
2           
3           
4           
results/FullProperty

In [6]:
# очистка файла
f = open('results/FullProperty.txt', 'w')
f.close()